In [3]:
import cv2
import matplotlib.pyplot as plt
import time
import os
import numpy as np


In [1]:
pip install matplotlib

     ---------------------------------------- 7.2/7.2 MB 1.7 MB/s eta 0:00:00
     -------------------------------------- 161.3/161.3 kB 9.4 MB/s eta 0:00:00
     ---------------------------------------- 55.4/55.4 kB 1.4 MB/s eta 0:00:00
     -------------------------------------- 965.4/965.4 kB 3.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
def detect_face2(img):
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    face_cascade=cv2.CascadeClassifier('')
    faces=face_cascade.detectMultiScale(gray,scaleFactor=1.1,minNeighbors=20);

    grays=[]
    face_cood=[]
    if(len(faces)==0):
        return None,None
    else:
        for face in faces:
            x,y,w,h=face
            grays.append(gray[y:y+w,x:x+h])
            face_cood.append(face)
    return grays,face_cood

def prepare_training_data(data_folder_path):
    dirs=os.listdir(data_folder_path)
    faces=[]
    labels=[]
    for dir_name in dirs:
        if not dir_name.startswith("s"):
            continue;
        label=int(dir_name.replace("s",""))
        dir_path=data_folder_path + "/"+dir_name
        images_names=os.listdir(dir_path)
        for image_name in images_names:
            image_path=dir_path+"/"+image_name
            image=cv2.imread(image_path)
            cv2.imshow("training on image...",image)
            cv2.waitKey(100)
            face,rect=detect_face2(image)
            if face is not None:
                face=face[0]
                rect=rect[0]
                faces.append(face)
                labels.append(label)
            cv2.destroyAllWindows()
            cv2.waitKey(1)
            cv2.destroyAllWindows()
    return faces,labels 

print("Preparing data...")
faces, labels = prepare_training_data(r"training")
print("Total numberof faces : ", len(faces))
print("Total number of labels : ",len(labels))
#we use LBP face recognizer and we feed the faces along with corresponding labels to it for...
face_recognizer = cv2.face.LBPHFaceRecognizer_create()
labels=np.array(labels)
face_recognizer.train(faces,labels) 

#this function draws the bounding box around the detected face in the image
def draw_rectangle(img, rect):
  (x, y, w, h) = rect
  cv2.rectangle(img, (x,y), (x+w,y+h), (0, 25, 0),2)

#this function is used to display the name over the bounding box
def draw_text(img, text, x, y):
  cv2.putText(img, text, (x,y), cv2.FONT_HERSHEY_PLAIN, 1.5, (0,255,0),2)

#the name of the subjects that is people can be placed here according to folders

# subject=["","harry","ankit","abhi"]
subject["","rahul"]

def predict(test_img):
  img=test_img.copy()
  faces, rects = detect_face2(img)
  if faces is None:
    return test_img
  else:
    #we use the for loop to loop over multiple faces detected in an image
    for face, rect in zip(faces,rects):
      label=face_recognizer.predict(face)
      label_text = subject[int(label[0])]

      draw_rectangle(img,rect)
      draw_text(img, label_text, rect[0], rect[1]-5)
  return img

#this function uses your webcam and takes frames from it 
#and feeds those frames one by one to the face recognizer
def startcam():
    capture = cv2.VideoCapture(0)
    while True:
      stime = time.time()
      ret, frame = capture.read(0)

      frame=cv2.flip(frame, 1)
      if ret:
        predicted_img1=predict(frame)
        cv2.imshow('frame',predicted_img1)

        print('FPS {:.1f}'.format(1 / (time.time() - stime)))
      if cv2.waitKey(50) & 0xFF == ord('q'):
        break
      capture.release()
      cv2.destroyAllWindows()

startcam()
